# Camera Caliibration

This notebook would give an overview on how to 

Perform Camera Calibration for a Generic Camera

In [1]:
import pathlib, os, pathlib

work_dir = os.path.abspath(os.getcwd())
work_dir_parent = pathlib.Path(work_dir).parent.absolute()

calib_pic_dir = os.path.join(work_dir_parent,'calibration_pics')

os.makedirs(calib_pic_dir, exist_ok=True)

In [2]:
import cv2
import numpy as np
from datetime import datetime

## Using USB Camera or PC Camera

Collect 15 or more images with the chessboard

Press "c" to capture image
Press "q to quit

In [3]:
frameWidth = 640
frameHeight = 480

cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
while True:
    success, img = cap.read()
    cv2.imshow("Live Capture", img)
    
    # process image here
    
    key  = cv2.waitKey(1)
    # press q to exit
    if key & 0xFF == ord('q'):
        break

    if key == ord("c"):
        now = datetime.now().strftime("%Y-%m-%d-%H%M%S")
        cv2.imwrite(calib_pic_dir+f'/{now}.jpg', img)

cv2.destroyAllWindows()
cap.release()

In [33]:
# set variables

dirpath = calib_pic_dir     # directory of the images
square_size = 0.025     # square size in meters
width = 9               # number of squares in width
height = 6              # number of squares in height
visualize = True        # visualize process



# perform calibration
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
objp = np.zeros((height*width, 3), np.float32)
objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)
objp = objp * square_size
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.
images = os.listdir(dirpath)
for fname in images:
    img = cv2.imread(os.path.join(dirpath, fname))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (width, height), None)
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (width, height), corners2, ret)
    if visualize:
        cv2.imshow('img',img)
        cv2.waitKey(0)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
np.save("calibration_matrix", mtx)
np.save("distortion_coefficients", dist)

In [34]:
ret

0.3119738146333606

In [35]:
mtx

array([[567.37024762,   0.        , 317.05813065],
       [  0.        , 567.86353292, 213.93060575],
       [  0.        ,   0.        ,   1.        ]])

In [36]:
dist

array([[ 0.14427071, -0.55556789, -0.01507111, -0.00228134,  0.74420082]])

In [39]:
# try undistorting image

test_img = cv2.imread("/home/richard/realsense_workshop/day_2_am/calibration_pics/2024-08-25-220749.jpg")

h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]


cv2.imshow("Distorted", img)
cv2.imshow("Undistorted", dst)

cv2.waitKey(0)
cv2.destroyAllWindows()